In [1]:
import polars as pl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 데이터 로드
file_path = "../food/*.parquet"

raw_df = (
    pl.scan_parquet(file_path)
    .select(["admi_cty_no", "card_tpbuz_nm_2", "hour", "sex", "age", "day", "cnt"])
    .filter((pl.col("cnt") > 0) & (pl.col("age").is_between(2, 6)))
    .collect(engine="streaming")
    .sample(n=1000000, seed=42)
)

# 안전한 매핑 함수
def get_mapping(col_name):
    unique_vals = raw_df[col_name].cast(pl.String).unique().to_list()
    mapping = {val: i for i, val in enumerate(unique_vals)}
    return mapping, len(unique_vals)

area_map, n_areas = get_mapping("admi_cty_no")
biz_map, n_classes = get_mapping("card_tpbuz_nm_2")
sex_map, n_sex = get_mapping("sex")

# 매핑 적용 (age는 2~6이므로 2를 빼서 0~4로 변환)
train_df = raw_df.with_columns([
    pl.col("admi_cty_no").cast(pl.String).replace(area_map).cast(pl.Int64),
    pl.col("card_tpbuz_nm_2").cast(pl.String).replace(biz_map).cast(pl.Int64),
    pl.col("sex").cast(pl.String).replace(sex_map).cast(pl.Int64),
    (pl.col("age").cast(pl.Int64) - 2).alias("age_idx"),
    pl.col("hour").cast(pl.Float32),
    pl.col("day").cast(pl.Int64)
])

In [2]:
class FoodRecommender(nn.Module):
    def __init__(self, n_areas, n_classes, n_sex):
        super(FoodRecommender, self).__init__()
        self.area_emb = nn.Embedding(n_areas, 16)
        self.sex_emb = nn.Embedding(n_sex, 2)
        self.age_emb = nn.Embedding(5, 4)  # 20대~60대 (0~4)
        self.day_emb = nn.Embedding(8, 4)  # 0~7
        
        # 입력 차원: 16(area) + 2(sex) + 4(age) + 4(day) + 1(hour) = 27
        self.fc = nn.Sequential(
            nn.Linear(27, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, area, hour, sex, age, day):
        x = torch.cat([
            self.area_emb(area),
            self.sex_emb(sex),
            self.age_emb(age),
            self.day_emb(day),
            hour.unsqueeze(1)
        ], dim=1)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodRecommender(n_areas, n_classes, n_sex).to(device)

class FoodDataset(Dataset):
    def __init__(self, df):
        self.area = df["admi_cty_no"].to_numpy()
        self.hour = df["hour"].to_numpy()
        self.sex = df["sex"].to_numpy()
        self.age = df["age_idx"].to_numpy()
        self.day = df["day"].to_numpy()
        self.label = df["card_tpbuz_nm_2"].to_numpy()

    def __len__(self): return len(self.label)
    def __getitem__(self, idx):
        return (torch.tensor(self.area[idx]), torch.tensor(self.hour[idx]),
                torch.tensor(self.sex[idx]), torch.tensor(self.age[idx]),
                torch.tensor(self.day[idx]), torch.tensor(self.label[idx]))

loader = DataLoader(FoodDataset(train_df), batch_size=2048, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    loss_sum = 0
    for area, hour, sex, age, day, label in loader:
        area, hour, sex, age, day, label = area.to(device), hour.to(device), sex.to(device), age.to(device), day.to(device), label.to(device)
        
        optimizer.zero_grad()
        output = model(area, hour, sex, age, day)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    print(f"Epoch {epoch+1}, Loss: {loss_sum/len(loader):.4f}")

Epoch 1, Loss: 2.1871


KeyboardInterrupt: 

In [ ]:
# # 모델 저장하는 코드 (사용하려면 ctrl+/로 주석 제거하면 실행됨)

# import torch

# # 모델 저장 경로 설정
# save_path = "food_recommender.pth"

# # 모델의 state_dict(가중치) 저장
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'n_areas': n_areas,
#     'n_classes': n_classes,
#     'n_sex': n_sex,
#     'biz_map': biz_map,
#     'sex_map': sex_map,
#     'area_map': area_map
# }, save_path)

# print(f"✅ 모델과 매핑 사전이 '{save_path}'에 안전하게 저장되었습니다.")

In [ ]:
# # 모델 읽어오는 코드 (사용하려면 ctrl+/로 주석 제거하면 실행됨)

# # 1. 파일 불러오기
# checkpoint = torch.load("food_recommender.pth")

# # 2. 저장했던 설정값으로 모델 다시 생성
# loaded_model = FoodRecommender(
#     checkpoint['n_areas'], 
#     checkpoint['n_classes'], 
#     checkpoint['n_sex']
# ).to(device)

# # 3. 가중치 입히기
# loaded_model.load_state_dict(checkpoint['model_state_dict'])
# loaded_model.eval() # 추론 모드로 설정

# # 4. 매핑 사전들도 복구
# biz_map = checkpoint['biz_map']
# sex_map = checkpoint['sex_map']
# area_map = checkpoint['area_map']
# inv_biz_map = {v: k for k, v in biz_map.items()}

# print("✅ 모델 불러오기 성공! 이제 바로 verify_all_recommendations 함수를 쓸 수 있습니다.")

In [ ]:
# 입력된 값을 기준으로 답변

import torch
import torch.nn.functional as F

# 1. 역매핑 사전 및 필수 함수 확인 (유지)
try:
    inv_biz_map = {v: k for k, v in biz_map.items()}
except NameError:
    print("❌ biz_map이 정의되지 않았습니다.")

def get_hour_idx(real_hour):
    if 0 <= real_hour < 7: return 0
    elif 7 <= real_hour < 9: return 1
    elif 9 <= real_hour < 11: return 2
    elif 11 <= real_hour < 13: return 3
    elif 13 <= real_hour < 15: return 4
    elif 15 <= real_hour < 17: return 5
    elif 17 <= real_hour < 19: return 6
    elif 19 <= real_hour < 21: return 7
    elif 21 <= real_hour < 23: return 8
    elif 23 <= real_hour < 24: return 9
    else: return 10

# 2. 전체 순위 확인 함수 (변환된 버전)
def verify_all_ranks(age_val, sex_val, day_val, real_hour):
    model.eval()
    with torch.no_grad():
        try:
            # 인덱스 변환
            d_idx = int(day_val) - 1
            s_idx = sex_map[str(sex_val)]
            ag_idx = int(age_val) - 2
            h_idx = get_hour_idx(real_hour)

            # 텐서 생성 및 GPU 이동
            area_t = torch.tensor([0], dtype=torch.long).to(device)
            sex_t = torch.tensor([s_idx], dtype=torch.long).to(device)
            age_t = torch.tensor([ag_idx], dtype=torch.long).to(device)
            day_t = torch.tensor([d_idx], dtype=torch.long).to(device)
            hour_t = torch.tensor([float(h_idx)], dtype=torch.float32).to(device)

            # 예측
            outputs = model(area_t, hour_t, sex_t, age_t, day_t)
            probs = F.softmax(outputs, dim=1)
            
            # [수정 포인트] 5개가 아니라 전체 클래스 개수(n_classes)만큼 가져옵니다.
            all_probs, all_indices = torch.topk(probs, n_classes)

            print(f"📊 [전체 카테고리 추천 순위]")
            print(f"상황: {age_val}0대 {sex_val} | 요일:{day_val} | 시간:{real_hour}시")
            print("=" * 55)
            print(f"{'순위':<5} | {'업종명':<18} | {'확률(%)'}")
            print("-" * 55)
            
            for i in range(n_classes):
                idx = all_indices[0][i].item()
                biz_nm = inv_biz_map.get(idx, f"알 수 없음({idx})")
                score = all_probs[0][i].item() * 100
                print(f"{i+1:>3}위  | {biz_nm:<18} | {score:>8.4f}%")
            print("=" * 55)
            
        except Exception as e:
            print(f"❌ 추론 오류: {e}")

# 3. 테스트 실행
verify_all_ranks(age_val=3, sex_val='F', day_val=1, real_hour=12)

# # 4. JSON 파일로 저장
# with open("category_recommendation_map.json", "w", encoding="utf-8") as f:
#     json.dump(recommendation_result, f, ensure_ascii=False, indent=4)

# print(f"\n✅ 총 {len(recommendation_result)}개 상황에 대한 전체 순위가 'category_recommendation_map.json'에 저장되었습니다.")

In [ ]:
# 로그 스케일로 변환하여 답변

import torch
import torch.nn.functional as F

def verify_all_ranks_final(age_val, sex_val, day_val, real_hour):
    model.eval()
    
    # 1. inv_biz_map 자동 복구 로직
    try:
        # biz_map이 메모리에 있는지 확인하고 역사전 생성
        current_inv_biz_map = {v: k for k, v in biz_map.items()}
    except NameError:
        print("❌ 오류: biz_map(업종 사전)이 메모리에 없습니다. 데이터 전처리 셀을 실행해주세요.")
        return

    with torch.no_grad():
        try:
            # 2. 인덱스 및 시간 변환
            d_idx = int(day_val) - 1  # 1~7 -> 0~6
            s_idx = sex_map[str(sex_val)]
            ag_idx = int(age_val) - 2 # 2~6 -> 0~4
            h_idx = get_hour_idx(real_hour)

            # 3. 텐서 생성 및 GPU 이동
            area_t = torch.tensor([0], dtype=torch.long).to(device)
            sex_t = torch.tensor([s_idx], dtype=torch.long).to(device)
            age_t = torch.tensor([ag_idx], dtype=torch.long).to(device)
            day_t = torch.tensor([d_idx], dtype=torch.long).to(device)
            hour_t = torch.tensor([float(h_idx)], dtype=torch.float32).to(device)

            # 4. 모델 추론 및 확률 계산
            outputs = model(area_t, hour_t, sex_t, age_t, day_t)
            probs = F.softmax(outputs, dim=1)
            
            # 전체 업종 개수만큼 추출
            current_n_classes = outputs.shape[1] 
            all_probs, all_indices = torch.topk(probs, current_n_classes)

            print(f"📊 [전체 업종 추천 순위 - 로그 가중치 반영]")
            print(f"입력: {age_val}0대 {sex_val} | 요일:{day_val} | 시간:{real_hour}시")
            print("=" * 60)
            print(f"{'순위':<5} | {'업종명':<20} | {'확률(%)'}")
            print("-" * 60)
            
            for i in range(current_n_classes):
                idx = all_indices[0][i].item()
                biz_nm = current_inv_biz_map.get(idx, f"Unknown({idx})")
                score = all_probs[0][i].item() * 100
                print(f"{i+1:>3}위  | {biz_nm:<20} | {score:>8.4f}%")
            print("=" * 60)
            
        except Exception as e:
            print(f"❌ 추론 중 오류 발생: {e}")

# 실행
verify_all_ranks_final(age_val=3, sex_val='F', day_val=1, real_hour=12)

# # 4. JSON 파일로 저장
# with open("category_recommendation_map.json", "w", encoding="utf-8") as f:
#     json.dump(recommendation_result, f, ensure_ascii=False, indent=4)

# print(f"\n✅ 총 {len(recommendation_result)}개 상황에 대한 전체 순위가 'category_recommendation_map.json'에 저장되었습니다.")